In [1]:
!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.1 MB/s eta 0:00:0000:0100:01


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, PreTrainedModel, LlamaPreTrainedModel, LlamaModel, AutoConfig

from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from huggingface_hub import notebook_login
import torch
from transformers.modeling_outputs import SequenceClassifierOutputWithPast

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
dataset_name = "multi_nli"

In [5]:
notebook_login()
#

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import torch.nn as nn
import bitsandbytes as bnb
from torch.nn import CrossEntropyLoss
class CLF(LlamaPreTrainedModel):
    def __init__(self, config, num_labels):
        super().__init__(config)
        self.num_labels = num_labels
        self.model = LlamaModel(config)
        self.score = nn.Linear(config.hidden_size, self.num_labels, bias=False)
        self.post_init()
    
    def forward(self, input_ids=None,
                attention_mask=None,
                labels=None,
                position_ids=None,
                past_key_values=None,
                inputs_embeds=None,
                use_cache=None,
                output_attentions=None,
                output_hidden_states=None,
                return_dict=None):        
        
        transformers_outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                inputs_embeds=inputs_embeds,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict)

        hidden_states = transformers_outputs.last_hidden_state
        logits = self.score(hidden_states)
        
        batch_size = input_ids.shape[0]
        
        sequence_lengths = torch.eq(input_ids, self.config.pad_token_id).int().argmax(-1) - 1
        sequence_lengths = sequence_lengths % input_ids.shape[-1]
        sequence_lengths = sequence_lengths.to(logits.device)
        pooled_logits = logits[torch.arange(batch_size, device=logits.device), sequence_lengths]
        
        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(pooled_logits.view(-1, self.num_labels), labels.view(-1))
        
        if not return_dict:
            output = (pooled_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output
        
        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=pooled_logits,
            past_key_values=transformers_outputs.past_key_values,
            hidden_states=transformers_outputs.hidden_states,
            attentions=transformers_outputs.attentions,
        )

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)

In [9]:
num_labels = 3
id2label = {0: 'entailment', 1: 'contradiction', 2: 'neutral'}
label2id = {label: idx for idx, label in id2label.items()}

In [10]:
config = AutoConfig.from_pretrained(model_name, num_labels=len(id2label))
config.id2label = id2label
config.label2id = label2id
config.pad_token_id = tokenizer.pad_token_id
model = CLF.from_pretrained(
    model_name,
    config=config,
    num_labels=len(id2label),
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of CLF were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [12]:
print(model)

CLF(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm):

In [13]:
peft_config = LoraConfig(
    lora_alpha= 16,
    lora_dropout= 0.05,
    r= 16,
    bias="none",
    task_type="SEQ_CLS",
    target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj", "score"]
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 42,020,912 || all params: 7,547,023,456 || trainable%: 0.5568


In [15]:
dataset = load_dataset(dataset_name)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [16]:
model.get_memory_footprint()

5760475520

In [17]:
import random
def sample_5_percent(dataset):
    total_size = len(dataset)
    sample_size = max(1, 5 * total_size // 100)
    indices = random.sample(range(total_size), sample_size)
    return dataset.select(indices)

In [18]:
sampled_train_dataset = sample_5_percent(dataset['train'])
sampled_validation_dataset = sample_5_percent(dataset['validation_matched'])
sampled_train_dataset.shape, sampled_validation_dataset.shape

((19635, 10), (490, 10))

In [19]:
INSTRUCTION = "Decide whether the following statements are entailment, contradiction, or neutral."

In [20]:
def create_prompt(example, instruction):
    premise = example['premise']
    hypothesis = example['hypothesis']
    user_input = f"\nPremise: {premise}\nHypothesis: {hypothesis}\n"
    prompt = f"""<|start_header_id|>system<|end_header_id|> {instruction}<|eot_id|><|start_header_id|>user<|end_header_id|> Are these two statements entailment, contradiction, or neutral?: {user_input}<|eot_id|>"""
    return prompt

In [21]:
train_dataset = sampled_train_dataset.map(lambda example: {'prompt': create_prompt(example, INSTRUCTION)})

Map:   0%|          | 0/19635 [00:00<?, ? examples/s]

In [22]:
train_dataset['prompt'][0]

"<|start_header_id|>system<|end_header_id|> Decide whether the following statements are entailment, contradiction, or neutral.<|eot_id|><|start_header_id|>user<|end_header_id|> Are these two statements entailment, contradiction, or neutral?: \nPremise: What's more, they observe something called the minimal group effect.\nHypothesis: The minimal group effect is also observed.\n<|eot_id|>"

In [23]:
train_dataset

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'prompt'],
    num_rows: 19635
})

In [24]:
def preprocess_function(examples):
    outputs = tokenizer(examples['prompt'], truncation=True, max_length=None)
    return outputs

In [25]:
train_tokenized_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["promptID", "pairID", "premise", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre", "prompt"]).rename_column('label', 'labels')

Map:   0%|          | 0/19635 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [26]:
train_tokenized_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 19635
})

In [27]:
class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        batch = self.tokenizer.pad(
            features,
            padding=True,
            return_tensors="pt"
        )
        return batch

In [28]:
data_collator = CustomDataCollator(tokenizer)

In [29]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from accelerate import Accelerator

training_args=TrainingArguments(
        # num_train_epochs= 1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        logging_steps= 30,
        warmup_steps=5,
        max_steps=240,
        learning_rate=2e-5,
        weight_decay= 0.01,
        fp16= False,
        bf16= False,
        max_grad_norm= 0.3,
        group_by_length= True,
        optim="paged_adamw_8bit",
        output_dir= "/content/drive/MyDrive/weigths",
        report_to="none", 
)

# accelerator = Accelerator()
# model_with_clf, tokenizer, training_args = accelerator.prepare(model_with_clf, tokenizer, training_args)

trainer = Trainer(
    model=model,
    train_dataset=train_tokenized_dataset,
    args=training_args,
    data_collator=data_collator,
)

2024-06-01 21:14:07.753144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 21:14:07.753253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 21:14:07.905701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
max_steps is given, it will override any value given in num_train_epochs


In [30]:
model.train()
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,1.238900
60,1.126600
90,1.171900
120,1.121700
150,1.123400
180,1.114600
210,1.122800
240,1.100900


TrainOutput(global_step=240, training_loss=1.1401063760121664, metrics={'train_runtime': 3003.4363, 'train_samples_per_second': 1.279, 'train_steps_per_second': 0.08, 'total_flos': 1.31816723460864e+16, 'train_loss': 1.1401063760121664, 'epoch': 0.1955591770217967})

In [31]:
model.eval()

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): CLF(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128009)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (ba

In [32]:
validation_set = sampled_validation_dataset.map(lambda example: {'prompt': create_prompt(example, INSTRUCTION)})

Map:   0%|          | 0/490 [00:00<?, ? examples/s]

In [33]:
val_match= validation_set.map(preprocess_function, batched=True, remove_columns=["promptID", "pairID", "premise", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre", "prompt"]).rename_column('label', 'labels')

Map:   0%|          | 0/490 [00:00<?, ? examples/s]

In [34]:
val_match

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 490
})

In [35]:
from torch.utils.data import DataLoader

In [36]:
val_loader = DataLoader(val_match, batch_size=4, collate_fn=data_collator)

In [37]:
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [38]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [39]:
accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.3469
Precision: 0.1204
Recall: 0.3469
F1 Score: 0.1787


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
